In [ ]:
import sys
sys.path.append('../code')
from init_mooc_nb import *
init_notebook()

import itertools
import warnings
warnings.simplefilter("ignore", UserWarning)

# Pauli matrices
sigma0 = np.array([[1, 0], [0, 1]])
sigmax = np.array([[0, 1], [1, 0]])
sigmay = np.array([[0, -1j], [1j, 0]])
sigmaz = np.array([[1, 0], [0, -1]])


# Onsite and hoppings terms.
# We have two model now. par.model parameter
# Allows to choose between them. Valid models are 
# "BHZ and "QAH"

def onsite(site, par):
    if par.model == "BHZ":
        A, B, D, M = par.A/2, par.B, par.D, par.M
        output = ((M - 4 * B) * np.kron(sigma0, sigmaz) - 4 * D * np.kron(sigma0, sigma0) 
                  + par.field * site.pos[1] * np.kron(sigma0, sigma0))
    if par.model == "QAH":
        B, mu = par.B, par.mu
        output = (mu - 4*B) * sigmaz + par.field * site.pos[1] * sigma0
    return output


def hopx(site1, site2, par):
    A, B, D = par.A, par.B, par.D
    if par.model == 'BHZ':
        output = B * np.kron(sigma0, sigmaz) + D * np.kron(sigma0, sigma0) + 1j * A/2 * np.kron(sigmaz, sigmax)
    if par.model == 'QAH':
        output = B * sigmaz + 1j * A/2 * sigmax
    return output


def hopy(site1, site2, par):
    A, B, D = par.A, par.B, par.D
    if par.model == 'BHZ':
        output = B * np.kron(sigma0, sigmaz) + D * np.kron(sigma0, sigma0) - 1j * A/2 * np.kron(sigma0, sigmay)
    if par.model == 'QAH':
        output = B * sigmaz + 1j * A/2 * sigmay
    return output


def weak_hopping(site1, site2, par):
    if par.model == 'BHZ': return par.t_inter * np.eye(4)
    if par.model == 'QAH': return par.t_inter * np.eye(2)


#-----------------------systems----------------------------
def create_screw_system(L, W, H, xs=None, ys=None, ye=None, pbc=True):
    """ Create system with screw dislocation.
    
    Function creates system with a screw dislocation. 
    L, W, H are dimension of scattering region.
    L, W are dimension of cross section. 
    Leads are attached from top and bottom (0,0,1) direction.
    
    xs, ys, ye describes where disloaction is placed.
    """
    def shape(pos):
        (x, y, z) = pos
        return (0 <= x < L) and (0 <= y < W) and (0 <= z < H)
    
    def lead_shape(pos):
        (x, y, z) = pos
        return (0 <= x < L) and (0 <= y < W)

    # calling kwant
    lat = kwant.lattice.general(np.identity(3))
    sys = kwant.Builder()
    
    sym = kwant.TranslationalSymmetry((0, 0, 1))
    lead = kwant.Builder(sym)
    
    # scattering system 
    sys[lat.shape(shape, (0,0,0))] = onsite
    sys[kwant.HoppingKind((1,0,0), lat)] = hopx
    sys[kwant.HoppingKind((0,1,0), lat)] = hopy
    sys[kwant.HoppingKind((0,0,1), lat)] = weak_hopping
    
    # lead system
    lead[lat.shape(lead_shape, (0,0,0))] = onsite
    lead[kwant.HoppingKind((1,0,0), lat)] = hopx
    lead[kwant.HoppingKind((0,1,0), lat)] = hopy
    lead[kwant.HoppingKind((0,0,1), lat)] = weak_hopping
    
    # defining dislocation
    if xs != None:
        for y in range(ys, ye):
            for z in range(H):
                del sys[lat(xs+1,y,z), lat(xs,y,z)]
                
            del lead[lat(xs+1,y,0), lat(xs,y,0)]
            lead[lat(xs+1,y,z+1), lat(xs,y,z)] = hopx
            
        for y,z in itertools.product(list(range(ys, ye)), list(range(H - 1))):
            sys[lat(xs+1,y,z+1), lat(xs,y,z)] = hopx
            
    # adding periodic boundary conditions
    if pbc:
        for x in range(L):
            lead[lat(x,0,0), lat(x, W-1, 0)] = hopy
            for z in range(H):
                sys[lat(x,0,z), lat(x, W-1, z)] = hopy

        for y in range(W):
            lead[lat(0, y, 0), lat(L-1,y,0)] = hopx
            for z in range(H):
                sys[lat(0, y, z), lat(L-1,y,z)] = hopx
        
    sys.attach_lead(lead)
    sys.attach_lead(lead.reversed())

    return sys.finalized()


# 
def create_edge_dislocation_system(L, W, H, xs=None, ys=None, ye=None, pbc=True):
    """ Create system with edge dislocation.
    
    Function creates system with an edge dislocation. 
    L, W, H are dimension of scattering region.
    L, W are dimension of cross section. 
    Leads are attached from top and bottom (0,0,1) direction.
    
    xs, ys, ye describes where disloaction is placed.
    """    
    def shape(pos):
        (x, y, z) = pos
        return (0 <= x < L) and (0 <= y < W) and (0 <= z < H)
    
    def lead_shape(pos):
        (x, y, z) = pos
        return (0 <= x < L) and (0 <= y < W)
    
    # Calling kwant
    lat = kwant.lattice.general(np.identity(3))
    sys = kwant.Builder()
    
    sym = kwant.TranslationalSymmetry((0, 0, 1))
    lead = kwant.Builder(sym)
    
    # scattering system 
    sys[lat.shape(shape, (0,0,0))] = onsite
    sys[kwant.HoppingKind((1,0,0), lat)] = weak_hopping
    sys[kwant.HoppingKind((0,1,0), lat)] = hopy
    sys[kwant.HoppingKind((0,0,1), lat)] = hopx
    
    # lead system
    lead[lat.shape(lead_shape, (0,0,0))] = onsite
    lead[kwant.HoppingKind((1,0,0), lat)] = weak_hopping
    lead[kwant.HoppingKind((0,1,0), lat)] = hopy
    lead[kwant.HoppingKind((0,0,1), lat)] = hopx
    
    # defining disclocation
    if xs != None:
        for y in range(ys, ye):
            del lead[lat(xs, y, 0)]
            lead[lat(xs+1, y, 0), lat(xs-1, y, 0)] = weak_hopping
            
            for z in range(H):    
                del sys[lat(xs, y, z)]
                sys[lat(xs+1, y, z), lat(xs-1, y, z)] = weak_hopping
                      
    # periodic boundary conditions   
    if pbc:
        for x in range(L):
            lead[lat(x,0,0), lat(x, W-1, 0)] = hopy
            for z in range(H):
                sys[lat(x,0,z), lat(x, W-1, z)] = hopy

        for y in range(W):
            lead[lat(0, y, 0), lat(L-1,y,0)] = weak_hopping
            for z in range(H):
                sys[lat(0, y, z), lat(L-1,y,z)] = weak_hopping
            
    # attaching leads
    sys.attach_lead(lead)
    sys.attach_lead(lead.reversed())

    return sys.finalized()       


def get_densities(lead, momentum, param, sorting_mid=0.0):
    """ Calculate calculate density of states in lead at given momentum. """
    coord = np.array([lead.pos(i) for i in range(lead.graph.num_nodes)])
    xy = coord[coord[:, 2] == 0][:, :-1]
    indxs_xy = np.lexsort(xy.T)
    xy = xy[indxs_xy, :]

    h, t = lead.cell_hamiltonian(args=[param]), lead.inter_cell_hopping(args=[param])
    h_k = lambda k: h + t * np.exp(1j * k) + t.T.conj() * np.exp(-1j * k)

    vals, vecs = np.linalg.eigh(h_k(momentum))
    
    
    if param.model == 'BHZ': num_orbital = 4
    if param.model == 'QAH': num_orbital = 2
    
    densities = np.linalg.norm(vecs.reshape(-1, num_orbital, len(vecs)), axis=1)**2

    indxs = np.argsort(abs(vals - sorting_mid))
    vals = vals[indxs]
    densities = densities[:, indxs]
    densities = densities[indxs_xy, :]

    L, W = int(np.max(xy[:, 0]) + 1), int(np.max(xy[:, 1]) + 1)
    twod_densities = np.zeros((W,L,densities.shape[1]))

    for coord, val in zip(xy, densities):
        i,j = np.array(coord, dtype=int)
        twod_densities[j, i, :] = val
        
    return twod_densities, vals

#Table of Contents
* [Introduction: weak topological phases](#Introduction:-weak-topological-phases)
* [Crystallographic defects and topology](#Crystallographic-defects-and-topology)
* [The role of defect dimensionality](#The-role-of-defect-dimensionality)
* [The defect topological invariant](#The-defect-topological-invariant)
* [Electronic states in dislocations](#Electronic-states-in-dislocations)
* [Conclusions](#Conclusions)


# Introduction: weak topological phases

Taylor Hughes from the University of Illinois at Urbana-Champaign will describe the interplay between defects in crystals and weak topological insulators.

In [ ]:
MoocVideo("k3ZKCg7jtTs", src_location="7.2-intro")

As you see, there is a simple and universal connection between weak topological phases and the ability of defects to carry topologically protected states. Specifically, the topological invariant of a dislocation $\mathcal{Q}$, so the number of protected states that it carries, can be determined from the vector of weak topological invariants $\mathbf{\mathcal{Q}}_\textrm{weak}$ and the Burgers vector of the dislocation $\mathbf{B}$:

$$\mathcal{Q} = \mathbf{\mathcal{Q}}_\textrm{weak}\cdot\mathbf{B}$$

Let us now go through the main points that lead to this conclusion and try to reason why it has to be that way.

# Crystallographic defects and topology

There are many different types of [defects in crystals](http://en.wikipedia.org/wiki/Crystallographic_defect): vacancies, substitutions, grain boundaries, dislocations, and many more.

Let us first think what kinds of defects may be important for topology. Consider a vacancy for example:
![](figures/vacancy.svg)

To create it we need to remove a single unit atom (or all the atoms following one line). Can this type of defect carry a topologically protected state?

We know already that the topological protection requires a Hamiltonian that cannot be created locally. So to create a single Majorana bound state we need to create another one elsewhere. Removing an atom or a line of atoms only changes the system locally, so the *other* topologically protected state cannot appear anywhere.

Just a vacancy is therefore not interesting. What kinds of topological defects would work? The crystallographic defects leave nothing different since they leave the bulk Hamiltonian untouched far away from the defect core.
That means we need to do something nontrivial to the crystal so that it cannot be removed locally.

Examples of such defects are dislocations:

![](figures/burgers_vectors.png)
(By David Gabriel García Andrade (Own work) [Public domain], via Wikimedia Commons)

In order to create a dislocation we need to cut a crystal along one plane and displace all the atoms along that plane by the *Burgers vector*. This has to be done all the way to the crystal boundary (or infinity in an infinite crystal), and so a dislocation cannot be removed locally.

So, as Taylor Hughes explained, a dislocation can be detected infinitely far from its core by going around it and verifying that we don't return to the point of origin. We cannot remove a dislocation by just locally replacing some atoms. Therefore it may carry a topologically protected mode.

Unsurprisingly, the crystallographic defects that cannot be removed locally are called "topological", and we come to the first important conclusion:

> *Topological* crystallographic defects are the ones that may carry topologically protected modes.

This is a non-trivial observation, even though it sounds tautological. There are two different types of topology involved: the topology of the electronic modes and topology of the crystal.

# The role of defect dimensionality

When do topological defects carry the protected edge states?

Far away from the defect the bulk is homogeneous. So the appearance of the edge state must be encoded jointly in the defect properties as well as the bulk Hamiltonian. Also of course the appearance of this state must be controlled by a topological invariant, since the protected state cannot disappear without the closing of the bulk gap.

What kind of topological invariant can this be? Can a strong topological invariant create a protected edge state appearing at a defect?

In a sense we already know that it does. The crystal surface is a defect that breaks translational symmetry, and so it is a crystallographic defect. The strong topological invariant is the quantity that tells us if the bulk cannot be continuously deformed into vacuum, or equivalently it tells us if the surface can be smoothly removed.

In a $d$-dimensional bulk the strong invariant is responsible for appearance of a $d-1$-dimensional topologically protected state. This state can only be bound to a surface (the only $d-1$-dimensional topological defect). Defects of lower dimensionality can not be impacted by the strong invariant. (An example of such defect is precisely a dislocation as in the previous figure, which is a one-dimensional defect in a three-dimensional crystal.)

That's where the weak invariants come into play.

The first thing we know is the type of the protected state we can expect to appear at the defect. Its dimensionality $d_\textrm{egde}$ must match the dimensionality of the defect. Further we already know the dimensionality of the topological invariant that controls this state; it is the topological invariant in the dimension $d_\textrm{edge}+1$.

The topological invariants with dimensionality $d_{edge}+1$ form a vector or a tensor of the weak indices. The last question we need to answer is how do these weak indices tell us about what happens at the impurity.

# The defect topological invariant

We are almost ready to conclude the derivation of the criterion for appearance of the protected states in dislocations.

To see how the weak topological invariant relates to the number of states in the dislocation we can start from deforming a weak topological insulator into a set of disconnected planes, each carrying protected states. If there is a single state approaching the dislocation, like you see in the figure below, it cannot backscatter and must therefore continue through the dislocation core.

![](figures/dislocation_helical.svg)

(adapted from Cdang (Own work), via Wikimedia Commons, [CC BY-SA 3.0](http://creativecommons.org/licenses/by-sa/3.0).)

Counting the number and the orientation of the crystal planes approaching the core of the dislocation is just the Burgers vector. So the number of edge states entering the dislocation core is the Burgers vector times the number of states per crystal plane, and we arrive to our final conclusion:

$$\mathcal{Q} = \mathbf{\mathcal{Q}}_\textrm{weak}\cdot\mathbf{B}$$

Let's now test this idea and see if we can observe the protected dislocation states.

# Electronic states in dislocations

Now that we know the main concepts, let's apply them to concrete examples. Let's take two models for topological insulators that we already know and apply them to lattice systems with dislocations.

We will create a 3D weak topological insulators by stacking many layers 2D topological insulators along the $z$ direction. For the single layers, we will use the BHZ model (by the way, note that the lecture of today was given by the H of BHZ!) for a time-reversal invariant insulator and the square lattice model for the quantum Hall effect that we used in week 4. In this way we can study dislocations both with and without time reversal symmetry. In both cases, we take the hoppings between different layers to be relatively weak with respect to those within the same layer.

Let's start with a screw dislocation connecting two layers. The system looks like this:

In [ ]:
# System parameters
L = 5+1
W = 6+1
ws = 3
xs = 2
sys = create_screw_system(L, W, 2, xs=xs, ys=0, ye=W-ws, pbc=False)




fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1, projection='3d')

kwant.plot(sys, site_size=0.0, site_lw=0.01, hop_lw=0.025, ax=ax, num_lead_cells=0);

# ax.set_xlabel('x')
# ax.set_ylabel('y')
ax.set_xticks(())
ax.set_yticks(())
ax.set_zticks(())
ax.view_init(50,-110)


The Burgers' vector is parallel to $z$ and it has unit length - the dislocation connects neighboring layers.

The figure above only shows two layers, but we imagine that the system is repeated identically over the $z$ direction. Over the $x$ and $y$ directions it has periodic boundary conditions. Above we only show half of the dislocation.

In this system we can study the band structure along the $z$ direction, and what is the wave function of the corresponding states. Let's look at them:

In [ ]:
# System parameters
L = 10+1
W = 20+1
ws = 5
xs = 4
sys = create_screw_system(L, W, 5, xs=xs, ys=ws, ye=W-ws)

# Model parameters
parameters = {
    'BHZ': {'A': 1.0, 'B': 1.0, 'D': 0.0, 'M': 0.8}, 
    'QAH': {'A': 1.0, 'B': 1.0, 'D': 0.0, 'mu': 0.8}}

param = {}
param['QAH'] = SimpleNamespace(model='QAH', field=.01, t_inter=-.1, **parameters['QAH'])
param['BHZ'] = SimpleNamespace(model='BHZ', field=.005, t_inter=-.1, **parameters['BHZ'])
models = ['QAH', 'BHZ']

# Taking lead and specifing momenta
momenta = np.linspace(np.pi - np.pi/4, np.pi + np.pi/4, 51)
energies = {}

lead = sys.leads[0]
momentum = np.pi + .1
densities_and_vals = {}

# Calculating band structure and densities
for model in models:
    bands = kwant.physics.Bands(sys.leads[0], args=[param[model]])
    energies[model] = [bands(k) for k in momenta]
    densities_and_vals[model] = get_densities(lead, momentum, param[model], sorting_mid=.18)

# plotting
def plot(n, model='QAH'):
    fig = plt.figure(figsize=[9.5, 4], tight_layout=True)
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    ax1.set_color_cycle(['k'])

    densities, vals = densities_and_vals[model]
    # Plotting
    ax1.plot(momenta, energies[model])
    ax1.plot([momentum], vals[n], 'ro')
    ax1.plot([momentum, momentum], [-10, 10], '--')

    ax2.plot([xs+0.5, xs+0.5],[ws-.5, W-ws-.5], 'k--', lw=2)
    ax2.imshow(densities[:, :, n], cmap='gist_heat_r')#, interpolation='nearest')

    # Labels
    ax1.set_xlabel('$k$')
    ax1.set_ylabel('$E$')

    ax2.set_xlabel('$x$')
    ax2.set_ylabel('$y$')

    # Titles
    ax1.set_title("band structure")
    ax2.set_title("$|\psi|^2$");

    # Ticks
    evals = [-.5, 0, .5, 1]
    ax1.set_yticks(evals)
    ax1.set_yticklabels(["${0}$".format(i) for i in evals]);

    ax1.set_xticks([np.pi-np.pi/4, np.pi-np.pi/8, np.pi, np.pi+np.pi/8, np.pi+np.pi/4])
    ax1.set_xticklabels([r'$\pi-\frac{\pi}{4}$', '', r'$\pi$', '', r'$\pi+\frac{\pi}{4}$'])

    evals = [0, 5, 10]
    ax2.set_xticks(evals)
    ax2.set_xticklabels(["${0}$".format(i) for i in evals]);

    evals = [0, 10, 20]
    ax2.set_yticks(evals)
    ax2.set_yticklabels(["${0}$".format(i) for i in evals]);


    # Limits
    ax1.set_xlim(np.pi-np.pi/4, np.pi+np.pi/4)
    ax1.set_ylim(-.6,0.95);

    return fig

StaticInteract(plot, n=RangeWidget(0,6), model=DropDownWidget(['BHZ', 'QAH']))

You see that the band structure is gapless: because of the presence of the dislocation, there are states dispersing below the bulk gap along the $z$ direction.

A look on their wavefunction on the right panel shows that these low-energy states are localized in the $x$-$y$ plane around the end points of the dislocation (we show the wave function corresponding to the red dot on the band structure plot). On the other hand, when you look at the wave function of states above the gap, you see that they are spread out the whole $x$-$y$ plane.

The fundamental difference between the BHZ model case and the quantum anomalous Hall case is that in the former the gapless states at the dislocation are helical, in the latter they are chiral.

We can also look at an edge dislocation:

In [ ]:
# System parameters
L = 6+1
W = 6+1
ws = 3
xs = 3
sys = create_edge_dislocation_system(L, W, 2, xs=xs, ys=0, ye=W-ws, pbc=False)


fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1, projection='3d')

kwant.plot(sys, site_size=0.0, site_lw=0.01, hop_lw=0.025, ax=ax, num_lead_cells=0);

# ax.set_xlabel('x')
# ax.set_ylabel('y')
ax.set_xticks(())
ax.set_yticks(())
ax.set_zticks(())
ax.view_init(50,-110)

The Burgers vector is now along the $y$ direction, and it still has unit length. The band structure and the wave function plots show similar behavior.

In [ ]:
# System parameters
L = 10+1
W = 20+1
ws = 5
xs = 4
sys = create_edge_dislocation_system(L, W, 5, xs=xs, ys=ws, ye=W-ws)

# Model parameters
parameters = {
    'BHZ'     : { 'A': 1.0, 'B': 1.0, 'D': 0.0, 'M':  0.8}, 
    'QAH'     : { 'A': 1.0, 'B': 1.0, 'D': 0.0, 'mu': 0.8}}

param = {}
param['QAH'] = SimpleNamespace(model='QAH', field=.01, t_inter=-.1, **parameters['QAH'])
param['BHZ'] = SimpleNamespace(model='BHZ', field=.005, t_inter=-.1, **parameters['BHZ'])
models = ['QAH','BHZ']

# Taking lead and defining momenta
momenta = np.linspace(-np.pi/4, +np.pi/4, 51)
energies = {}

lead = sys.leads[0]
momentum = .1
densities_and_vals = {}

# Calculating band structure and densities
for model in models:
    bands = kwant.physics.Bands(sys.leads[0], args=[param[model]])
    energies[model] = [bands(k) for k in momenta]
    densities_and_vals[model] = get_densities(lead, momentum, param[model], sorting_mid=.0)
    
# plotting    
def plot(n, model='QAH'):
    fig = plt.figure(figsize=[9.5, 4], tight_layout=True)
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    ax1.set_color_cycle(['k'])

    densities, vals = densities_and_vals[model]
    
    # Plotting
    ax1.plot(momenta, energies[model])
    ax1.plot([momentum], vals[n], 'ro')
    ax1.plot([np.pi,np.pi], [-10,10], '--')

    ax2.plot([xs+0.5, xs+0.5, xs-0.5, xs-0.5, xs+0.5],[ws-.5, W-ws-.5, W-ws-.5, ws-.5, ws-.5], 'k-', lw=2)
    ax2.imshow(densities[:, :, n], cmap='gist_heat_r')#, interpolation='nearest')


    # Labels
    ax1.set_xlabel('$k$')
    ax1.set_ylabel('$E$')

    ax2.set_xlabel('$x$')
    ax2.set_ylabel('$y$')

    # Titles
    ax1.set_title("band structure")
    ax2.set_title("$|\psi|^2$");

    # Ticks
    evals = [-.5, 0, .5, 1]
    ax1.set_yticks(evals)
    ax1.set_yticklabels(["${0}$".format(i) for i in evals]);

    ax1.set_xticks([-np.pi/4, -np.pi/8, 0.0, np.pi/8, np.pi/4])
    ax1.set_xticklabels([r'$-\frac{\pi}{4}$', '', r'$0$', '', r'$\frac{\pi}{4}$'])

    evals = [0, 5, 10]
    ax2.set_xticks(evals)
    ax2.set_xticklabels(["${0}$".format(i) for i in evals]);

    evals = [0, 5, 10, 15, 20]
    ax2.set_yticks(evals)
    ax2.set_yticklabels(["${0}$".format(i) for i in evals]);


    # Limits
    ax1.set_xlim(-np.pi/4, np.pi/4)
    ax1.set_ylim(-.6,1.1);

    return fig

StaticInteract(plot, n=RangeWidget(0,6), model=DropDownWidget(['BHZ', 'QAH']))

In [ ]:
question = ("What would happen if, in both the above simulations, we changed the dislocation, "
            "making the Burgers vector twice as long?")

answers = ["The wave function would just spread out a bit more because the dislocation is larger.",
           "The number of gapless states would double for both models.",
           "The gapless states would be gapped out for both models.",
           "The dislocation would only have gapless states in the quantum anomalous Hall case, not for the BHZ model."]

explanation = ("Doubling the Burgers vector doubles the topological invariant in the $\mathbb{Z}$ case, "
               "and changes it from non-trivial to trivial in the $\mathbb{Z}_2$ case.")

MoocMultipleChoiceAssessment(question=question, answers=answers, correct_answer=3, explanation=explanation)

# Conclusions

In [ ]:
MoocVideo("MvcvJiZYSSk", src_location="7.2-summary")

Questions about what you just learned? Ask them below!

In [ ]:
MoocDiscussion("Questions", "Crystalline defects")